# Проект по курсу "Рекомендательные системы"

## "Дружеское пари". Класс моделей: матричные факторизации
  
Правила заполнения ноутбуков на авто-проверку:
- повторить окружение преподавателя
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```
- не добавлять новые импорты и не использовать дополнительные библиотеки. В противном случае ноутбук не пройдёт проверку и получит `0` баллов
- писать код только в ячейках с пометкой # YOUR CODE HERE, сразу после этой пометки
- не менять код преподавателя
- не добавлять новые ячейки
- следить, чтобы не было warning - они автоматом фейлят задание
- перед сдачей проверить, что весь ноутбук прогонятся от начала до конца и все тесты проходят

**В данном проекте вам нужно будет подбирать гипер-параметры моделей. Писать код для подбора гипер-параметров, использовать optuna и т.п. рекомендуем в отдельном ноутбуке.**

**Библиотеки implicit и lightfm не фиксируют random state при num_threads > 1. Если результат работы модели не сильно превышает необходимомый порог и рандом может опустить его ниже требуемого уровня, рекомендуем продолжить повышение качества модели: тюнинг гипер-параметров, подбор фичей, подбор метода обработки датасета**

## Импорты и данные

In [3]:
# pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0

# !pip install pandas==2.2.3 numpy==1.26.4 scipy==1.12.0 requests==2.32.3
# !pip install rectools[lightfm]==0.12.0 implicit==0.7.2
# !{sys.executable} -m pip install lightfm rectools[lightfm]==0.12.0

## !!!!!!!!!!!!!!!!!! ##

# Python 3.12.4 - использую kernel в Jupiter Notebook с Python 3.10, Python 3.12.4 - системная версия Python.

# import sys
# print(sys.version)
# 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]

In [4]:
!python -V

Python 3.12.4


In [5]:
# Убедитесь, что вы не добавляете новые импорты в ноутбук. Решение должно быть ограничено данными библиотеками

import warnings
warnings.simplefilter("ignore")

import implicit
import rectools
import pandas as pd
import numpy as np
import scipy
import requests

print(implicit.__version__)
print(rectools.__version__)
print(pd.__version__)
print(np.__version__)
print(scipy.__version__)
print(requests.__version__)

0.7.2
0.12.0
2.2.3
1.26.4
1.12.0
2.32.3


In [6]:
import os.path

from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
import os
import threadpoolctl
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

Если у вас нет данных, то используйте закомментированный код

In [8]:
# from tqdm.auto import tqdm
# import zipfile as zf

# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [9]:
data_path = os.environ.get("DATA_PATH")
if data_path is None:
    data_path = "data_original"  # ваш путь к данным до папки data_original включительно (поменяйте при необходимости)

In [10]:
interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

print(interactions.shape)
interactions.head(5)

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [11]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()
print(test.shape[0])
print(test[Columns.User].nunique())

306752
111240


## ImplicitALS (16 баллов)

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях. 

Правила пари: 
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [13]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE
    df = train.copy()

    # Базовый вес = логарифм длительности
    df[Columns.Weight] = np.log1p(df[Columns.Weight])

    # Учитываем процент просмотра
    df.loc[df["watched_pct"] < 50, Columns.Weight] *= 0.5
    df.loc[df["watched_pct"] >= 90, Columns.Weight] *= 1.2

    dataset = Dataset.construct(
        interactions_df=df,
        user_features_df=None,
        item_features_df=None,
    )
    
    # raise NotImplementedError()
    
    return dataset


config = {
    "cls": "ImplicitALSWrapperModel",
    # YOUR CODE HERE
    "model": {
        "cls": "implicit.als.AlternatingLeastSquares",
        "factors": 1,
        "iterations": 50,
        "regularization": 1, # заметил, что не оказывает значимого влияния
        "alpha": 1,
        "random_state": 42,
        "num_threads": 1,
        "use_gpu": False
    },
    "fit_features_together": True
    # raise NotImplementedError()
}

In [14]:
%%time
model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.052

0.07800849763744298
CPU times: total: 1min 38s
Wall time: 1min 51s


## SVD (16 баллов)

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид. 

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [16]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE
    df = train.copy()
    
    # бинарные взаимодействия
    df['weight'] = 1.0
    
    dataset = Dataset.construct(
        interactions_df=df,
        user_features_df=None,
        item_features_df=None,
    )
    
    # raise NotImplementedError()
    return dataset

config = {
    'cls': 'PureSVDModel',
    # YOUR CODE HERE
    'factors': 1
    # raise NotImplementedError()
}


In [17]:
%%time

model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'PureSVDModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.066

0.07913627149370503
CPU times: total: 37.9 s
Wall time: 32.5 s


## Dataset with features (8 баллов)

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [19]:
def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE
    df = train.copy()
    df[Columns.Weight] = np.log1p(df[Columns.Weight])
    df.loc[df["watched_pct"] < 50, Columns.Weight] *= 0.5
    df.loc[df["watched_pct"] >= 90, Columns.Weight] *= 1.2

    # item features
    item_features_frames = []
    
    # content_type
    item_features_frames.append(
        items.reindex(columns=[Columns.Item, "content_type"])
             .rename(columns={Columns.Item: "id", "content_type": "value"})
             .assign(feature="content_type")
    )
    
    # genres
    genres_df = items.assign(genre=items["genres"].str.split(", "))
    genres_df = genres_df.explode("genre")
    genres_df = genres_df.reindex(columns=[Columns.Item, "genre"])
    genres_df.columns = ["id", "value"]
    genres_df["feature"] = "genre"
    
    item_features_frames.append(genres_df)

    item_features_df = pd.concat(item_features_frames)
    
    # user features
    user_features_frames = []
    categorical_user_features = ["age", "sex", "income", "kids_flg"]
    for feature_name in categorical_user_features:
        feature_frame = users.reindex(columns=[Columns.User, feature_name])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature_name
        user_features_frames.append(feature_frame)

    user_features_df = pd.concat(user_features_frames)

    # Constructing используя sparse features

    dataset = Dataset.construct(
        interactions_df=df,
        user_features_df=user_features_df,
        item_features_df=item_features_df,
        cat_user_features=categorical_user_features,
        cat_item_features=["content_type", "genre"],
        make_dense_user_features=False,
        make_dense_item_features=False,
    )
    
    # raise NotImplementedError()
    return dataset

In [20]:
dataset_with_features = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features (20 баллов)

Собрав датасет с фичами, вы готовы к третьему раунду пари. 

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [22]:
config = {
    "cls": "ImplicitALSWrapperModel",
    # YOUR CODE HERE
    "model": {
        "cls": "implicit.als.AlternatingLeastSquares",
        "factors": 1,
        "iterations": 25,
        "regularization": 1, # заметил, что не оказывает значимого влияния
        "alpha": 1,
        "random_state": 42,
        "num_threads": 1,
        "use_gpu": False
    },
    "fit_features_together": True
}

In [23]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.073

0.07868281884003804
CPU times: total: 9min 13s
Wall time: 10min 20s


## LightFM with features (20 баллов)

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего лида, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [25]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! #
# Не получается использовать warp, k-OS WARP, BPR

# Lightfm 1.17 - kernel dies in Jupyter
# Kernel Restarting
# The kernel for ШАД МТС/RecSys/hse_project.ipynb appears to have died. It will restart automatically.

# Получилось исправить проблему только таким образом:

# "Maybe you can try without the loss='warp' parameter (using default logistic loss).
# If it works without the parameter it may be the same issue as I reported in #675".
# https://github.com/lyst/lightfm/issues/690

def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    # YOUR CODE HERE
    '''
    Пытался упростить датасет - не помогает.
    
    Lightfm 1.17 - kernel dies in Jupyter
    Kernel Restarting
    The kernel for ШАД МТС/RecSys/hse_project.ipynb appears to have died. It will restart automatically.
    
    Получилось исправить проблему только таким образом:
    
    "Maybe you can try without the loss='warp' parameter (using default logistic loss).
    If it works without the parameter it may be the same issue as I reported in #675".
    https://github.com/lyst/lightfm/issues/690
    '''
    df = train.copy()
    df[Columns.Weight] = np.log1p(df[Columns.Weight])
    df.loc[df["watched_pct"] < 50, Columns.Weight] *= 0.5
    df.loc[df["watched_pct"] >= 90, Columns.Weight] *= 1.2

    # item features
    item_features_frames = []
    
    # content_type
    item_features_frames.append(
        items.reindex(columns=[Columns.Item, "content_type"])
             .rename(columns={Columns.Item: "id", "content_type": "value"})
             .assign(feature="content_type")
    )
    
    # genres
    genres_df = items.assign(genre=items["genres"].str.split(", "))
    genres_df = genres_df.explode("genre")
    genres_df = genres_df.reindex(columns=[Columns.Item, "genre"])
    genres_df.columns = ["id", "value"]
    genres_df["feature"] = "genre"
    
    item_features_frames.append(genres_df)

    item_features_df = pd.concat(item_features_frames)
    
    # user features
    user_features_frames = []
    categorical_user_features = ["age", "sex", "income", "kids_flg"]
    for feature_name in categorical_user_features:
        feature_frame = users.reindex(columns=[Columns.User, feature_name])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature_name
        user_features_frames.append(feature_frame)

    user_features_df = pd.concat(user_features_frames)

    # Constructing используя sparse features

    dataset = Dataset.construct(
        interactions_df=df,
        user_features_df=user_features_df,
        item_features_df=item_features_df,
        cat_user_features=categorical_user_features,
        cat_item_features=["content_type", "genre"],
        make_dense_user_features=False,
        make_dense_item_features=False,
    )
    
    return dataset


config = {
    'cls': 'LightFMWrapperModel',
    'verbose': 1,
    'epochs': 100,
    'num_threads': 1,
    'model': {
        'cls': 'LightFM',
        'no_components': 1,
        'learning_schedule': 'adagrad',
        'loss': 'logistic',
        'learning_rate': 0.03,
        'item_alpha': 0.01,
        'user_alpha': 0.01,
        'random_state': 32,
    }
}

In [26]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'LightFMWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.08

Epoch: 100%|█████████████████████████████████████████████████████████████████████████| 100/100 [15:48<00:00,  9.49s/it]


0.00021040340894195982


AssertionError: 